# Import Packages

In [63]:
import mysql.connector
from mysql.connector import errorcode
import config
import requests
import json
import pandas as pd
from urllib.request import urlopen
from json import loads

# Define Helper Functions

In [4]:
def get_educationdata(first_url):
    all_data = []
    response = urlopen(first_url)
    data = loads(response.read())
    count = len(data['results'])
    all_data.append(data['results'])
    i = len(all_data)

    print('expecting ' + str(data['count']) + ' rows of data')
    while count < data['count']:
        if data['next'] is not None:
            try:
                response = urlopen(data['next'])
                data = loads(response.read())
                count += len(data['results'])
                all_data.append(data['results'])
                i += 1
                print('appended page ' + str(i))
                
            except:
                print('failed to append page ' + str(i))
    return all_data

In [5]:
def pages_to_one_df(data):
    data_dfs = []
    for page in range(0, len(data)):
        data_dfs.append(pd.DataFrame(data[page]))

    return pd.concat(data_dfs).reset_index(drop = True)

In [35]:
def data_to_tuples(data):
    data_tup = []
    for row in data.values:
        data_tup.append(tuple(row))
    return data_tup

In [72]:
def insert_data(data_tup, tbl_name, columns):
    cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = 'colleges',
    buffered = True
    )
    cursor = cnx.cursor()
    
    len_col = len(columns)
    col_names = ', '.join(columns)
    num_col = '%s, '*len_col
    insert_statement = f"INSERT INTO {tbl_name} ({col_names}) VALUES ({num_col.strip(', ')})"
    print(insert_statement)
    cursor.executemany(insert_statement, data_tup)
    cnx.commit()
    cnx.close()
    return 'inserted data'

# Get Directory Info for Degree-Granting Institutions

In [10]:
url = "https://educationdata.urban.org/api/v1/college-university/ipeds/directory/2016"
ipeds_dir_data = get_educationdata(url)
ipeds_dir_df = pages_to_one_df(ipeds_dir_data)

expecting 7521 rows of data
appended page 2
appended page 3
appended page 4
appended page 5
appended page 6
appended page 7
appended page 8


In [8]:
ipeds_dir_df.shape

(7521, 76)

In [73]:
ipeds_dir_columns = ['unitid', 'year', 'inst_name', 'state_abbr', 'region',
                     'urban_centric_locale', 'inst_status', 'currently_active_ipeds', 
                     'degree_granting', 'open_public', 'title_iv_indicator', 
                     'postsec_public_active', 'postsec_public_active_title_iv', 
                     'year_deleted', 'inst_control', 'institution_level', 'inst_category', 
                     'inst_size', 'sector', 'primarily_postsecondary', 'hbcu', 
                     'hospital', 'medical_degree', 'tribal_college', 'land_grant', 
                     'offering_highest_degree', 'offering_highest_level', 'offering_undergrad', 
                     'offering_grad', 'cc_basic_2015', 'cc_instruc_undergrad_2015', 
                     'cc_enroll_2015', 'cc_size_setting_2015']
ipeds_dir = ipeds_dir_df[ipeds_dir_columns]

In [74]:
ipeds_dir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7521 entries, 0 to 7520
Data columns (total 33 columns):
unitid                            7521 non-null int64
year                              7521 non-null int64
inst_name                         7521 non-null object
state_abbr                        7521 non-null object
region                            7521 non-null int64
urban_centric_locale              7521 non-null int64
inst_status                       7521 non-null object
currently_active_ipeds            7521 non-null int64
degree_granting                   7521 non-null int64
open_public                       7521 non-null int64
title_iv_indicator                7521 non-null int64
postsec_public_active             7521 non-null int64
postsec_public_active_title_iv    7521 non-null int64
year_deleted                      7521 non-null int64
inst_control                      7521 non-null int64
institution_level                 7521 non-null int64
inst_category             

In [75]:
ipeds_dir = ipeds_dir.astype({'inst_status': 'int64'})
ipeds_dir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7521 entries, 0 to 7520
Data columns (total 33 columns):
unitid                            7521 non-null int64
year                              7521 non-null int64
inst_name                         7521 non-null object
state_abbr                        7521 non-null object
region                            7521 non-null int64
urban_centric_locale              7521 non-null int64
inst_status                       7521 non-null int64
currently_active_ipeds            7521 non-null int64
degree_granting                   7521 non-null int64
open_public                       7521 non-null int64
title_iv_indicator                7521 non-null int64
postsec_public_active             7521 non-null int64
postsec_public_active_title_iv    7521 non-null int64
year_deleted                      7521 non-null int64
inst_control                      7521 non-null int64
institution_level                 7521 non-null int64
inst_category              

In [76]:
ipeds_dir.inst_status.value_counts() # 1 = active institution
ipeds_dir = ipeds_dir[ipeds_dir['inst_status'] == 1]

In [77]:
ipeds_dir = ipeds_dir[ipeds_dir['offering_undergrad'] == 1]
ipeds_dir = ipeds_dir[ipeds_dir['degree_granting'] == 1]

In [78]:
ipeds_dir.region.value_counts()

5    1124
2     661
8     620
3     616
6     446
4     435
1     241
7     161
9      99
0       5
Name: region, dtype: int64

## Insert into MySQL Table

In [80]:
ipeds_dir_tuple = data_to_tuples(ipeds_dir)
insert_data(ipeds_dir_tuple, 'ipeds_dir', ipeds_dir_columns)

INSERT INTO ipeds_dir (unitid, year, inst_name, state_abbr, region, urban_centric_locale, inst_status, currently_active_ipeds, degree_granting, open_public, title_iv_indicator, postsec_public_active, postsec_public_active_title_iv, year_deleted, inst_control, institution_level, inst_category, inst_size, sector, primarily_postsecondary, hbcu, hospital, medical_degree, tribal_college, land_grant, offering_highest_degree, offering_highest_level, offering_undergrad, offering_grad, cc_basic_2015, cc_instruc_undergrad_2015, cc_enroll_2015, cc_size_setting_2015) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)


'inserted data'

# Get Institutional Characteristics

In [81]:
url = "https://educationdata.urban.org/api/v1/college-university/ipeds/institutional-characteristics/2016/"
ipeds_inst_char = get_educationdata(url)
ipeds_inst_char_df = pages_to_one_df(ipeds_inst_char)

expecting 7521 rows of data
appended page 2
appended page 3
appended page 4
appended page 5
appended page 6
appended page 7
appended page 8


In [97]:
ipeds_inst_char_columns = ['unitid', 'year', 'inst_affiliation', 'primary_public_control', 
                           'occupational_prog_offered', 'academic_prog_offered', 
                           'cont_prof_prog_offered', 'avocational_prog_offered', 
                           'remedial_prog_offered', 'secondary_prog_offered', 'assoc_offered', 
                           'bach_offered', 'postbac_cert_offered', 'masters_offered', 
                           'doctors_research_offered', 'doctors_professional_offered', 
                           'doctors_other_offered', 'enrolled_undergrad_fulltime', 
                           'remedial_services', 'academic_counseling', 'employment_services', 
                           'placement_services', 'oncampus_daycare', 'oncampus_housing', 
                           'oncampus_required', 'meal_plan', 'typical_room_charge', 
                           'typical_board_charge', 'room_board_charge', 'tuition_varies', 
                           'alt_tuition', 'religious_affiliation', 'ap_credit', 'rotc', 
                           'study_abroad', 'weekend_evening_college', 'disability_indicator', 
                           'yellow_ribbon_program', 'veteran_point_of_contact', 
                           'student_veteran_organization', 'no_vet_services', 'library_physical', 
                           'library_digital', 'library_trained_staff', 'library_shared', 
                           'no_library', 'calendar_system']
ipeds_inst_char = ipeds_inst_char_df[ipeds_inst_char_columns]

In [104]:
# Replace nan values with None
# Check if in directory
ipeds_inst_char = ipeds_inst_char.where((pd.notnull(ipeds_inst_char)), None)
ipeds_inst_char = ipeds_inst_char[ipeds_inst_char['unitid'].isin(ipeds_dir['unitid'])]

In [107]:
ipeds_inst_char.info() # looks like ~200 data points may be missing most data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4408 entries, 0 to 7339
Data columns (total 47 columns):
unitid                          4408 non-null int64
year                            4408 non-null int64
inst_affiliation                4261 non-null object
primary_public_control          4261 non-null object
occupational_prog_offered       4261 non-null object
academic_prog_offered           4261 non-null object
cont_prof_prog_offered          4261 non-null object
avocational_prog_offered        4261 non-null object
remedial_prog_offered           4261 non-null object
secondary_prog_offered          4261 non-null object
assoc_offered                   4261 non-null object
bach_offered                    4261 non-null object
postbac_cert_offered            4261 non-null object
masters_offered                 4261 non-null object
doctors_research_offered        4261 non-null object
doctors_professional_offered    4261 non-null object
doctors_other_offered           4261 non-null o

## Insert into MySql Table

In [112]:
ipeds_inst_char_tuple = data_to_tuples(ipeds_inst_char)
insert_data(ipeds_inst_char_tuple, 'ipeds_inst_char', ipeds_inst_char_columns)

INSERT INTO ipeds_inst_char (unitid, year, inst_affiliation, primary_public_control, occupational_prog_offered, academic_prog_offered, cont_prof_prog_offered, avocational_prog_offered, remedial_prog_offered, secondary_prog_offered, assoc_offered, bach_offered, postbac_cert_offered, masters_offered, doctors_research_offered, doctors_professional_offered, doctors_other_offered, enrolled_undergrad_fulltime, remedial_services, academic_counseling, employment_services, placement_services, oncampus_daycare, oncampus_housing, oncampus_required, meal_plan, typical_room_charge, typical_board_charge, room_board_charge, tuition_varies, alt_tuition, religious_affiliation, ap_credit, rotc, study_abroad, weekend_evening_college, disability_indicator, yellow_ribbon_program, veteran_point_of_contact, student_veteran_organization, no_vet_services, library_physical, library_digital, library_trained_staff, library_shared, no_library, calendar_system) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s

'inserted data'

# Get Enrollment Data

In [223]:
url = "https://educationdata.urban.org/api/v1/college-university/ipeds/admissions-enrollment/2016/"
ipeds_adm_enroll = get_educationdata(url)
ipeds_adm_enroll_df = pages_to_one_df(ipeds_adm_enroll)

expecting 22563 rows of data
appended page 2
appended page 3
appended page 4
appended page 5
appended page 6
appended page 7
appended page 8
appended page 9
appended page 10
appended page 11
appended page 12
appended page 13
appended page 14
appended page 15
appended page 16
appended page 17
appended page 18
appended page 19
appended page 20
appended page 21
appended page 22
appended page 23


In [224]:
# Replace numerical sex var with string representation
ipeds_adm_enroll_df = ipeds_adm_enroll_df.drop(['fips', 'year'], axis = 1)
ipeds_adm_enroll_df = ipeds_adm_enroll_df.replace({'sex': {1: 'M', 2: 'F', 99: 'N'}})

# Pivot dataframe
ipeds_adm_enroll_piv = ipeds_adm_enroll_df.pivot(index = 'unitid', columns = 'sex')
ipeds_adm_enroll_piv.reset_index(inplace = True)
ipeds_adm_enroll_piv.columns = ['_'.join(col).strip('_') for col in ipeds_adm_enroll_piv.columns.values]

In [225]:
# Replace nan values with None
# Check if in directory
ipeds_adm_enroll_piv = ipeds_adm_enroll_piv.where((pd.notnull(ipeds_adm_enroll_piv)), None)
ipeds_adm_enroll_piv = ipeds_adm_enroll_piv[ipeds_adm_enroll_piv['unitid'].isin(ipeds_dir['unitid'])] 

In [226]:
ipeds_adm_enroll_piv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4408 entries, 0 to 7339
Data columns (total 16 columns):
unitid                     4408 non-null int64
number_admitted_F          1833 non-null object
number_admitted_M          1849 non-null object
number_admitted_N          1897 non-null object
number_applied_F           1906 non-null object
number_applied_M           1906 non-null object
number_applied_N           1907 non-null object
number_enrolled_ft_F       1822 non-null object
number_enrolled_ft_M       1832 non-null object
number_enrolled_ft_N       1892 non-null object
number_enrolled_pt_F       1479 non-null object
number_enrolled_pt_M       1444 non-null object
number_enrolled_pt_N       1490 non-null object
number_enrolled_total_F    1830 non-null object
number_enrolled_total_M    1840 non-null object
number_enrolled_total_N    1895 non-null object
dtypes: int64(1), object(15)
memory usage: 585.4+ KB


## Insert into MySQL Table

In [228]:
ipeds_adm_enroll_tuple = data_to_tuples(ipeds_adm_enroll_piv)
insert_data(ipeds_adm_enroll_tuple, 'ipeds_adm_enroll', ipeds_adm_enroll_piv.columns)

INSERT INTO ipeds_adm_enroll (unitid, number_admitted_F, number_admitted_M, number_admitted_N, number_applied_F, number_applied_M, number_applied_N, number_enrolled_ft_F, number_enrolled_ft_M, number_enrolled_ft_N, number_enrolled_pt_F, number_enrolled_pt_M, number_enrolled_pt_N, number_enrolled_total_F, number_enrolled_total_M, number_enrolled_total_N) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)


'inserted data'

# Get Scorecard Student Characteristics

In [206]:
url = "https://educationdata.urban.org/api/v1/college-university/scorecard/student-characteristics/2016/aid-applicants/"
sc_stud_char = get_educationdata(url)
sc_stud_char_df = pages_to_one_df(sc_stud_char)

expecting 6976 rows of data
appended page 2
appended page 3
appended page 4
appended page 5
appended page 6
appended page 7


In [235]:
sc_stud_char_df.describe()

,count_family_income,count_family_income_dependent,count_family_income_independent,count_parent_ed,count_total_fafsa_applicants,count_valid_dep_status,dependent_highincome1_pct,dependent_highincome2_pct,dependent_highincome_pct,dependent_income_mean,...,midincome1_pct,midincome2_pct,midincome_pct,opeid6,parents_highest_ed_high_school,parents_highest_ed_middle_school,parents_highest_ed_postsecondary,unitid,veteran_pct,year
count,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,1921.000000,6976.000000,...,6976.000000,6976.000000,5121.000000,6976.000000,6976.000000,6976.000000,6976.000000,6.976000e+03,6976.000000,6976.0
mean,3159.814220,1230.813217,1922.176892,2622.659404,3159.814220,3159.814220,-2.129866,-2.132989,0.254190,45254.752133,...,-0.368712,-0.668607,0.273947,16546.096474,-1.001577,-1.006737,-0.399943,1.852134e+06,-1.851927,2016.0
std,7091.258552,2442.708313,5849.503324,5873.126193,7091.258552,7091.258552,1.399859,1.403510,0.147374,26352.561264,...,1.173290,1.356935,0.064493,14369.921477,1.414871,1.416679,1.019815,6.932749e+06,1.464363,0.0
min,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,0.019750,-3.000000,...,-3.000000,-3.000000,0.065241,1002.000000,-3.000000,-3.000000,-3.000000,1.006540e+05,-3.000000,2016.0
25%,184.000000,55.000000,73.750000,144.000000,184.000000,184.000000,-3.000000,-3.000000,0.145604,27790.700000,...,0.106136,-3.000000,0.231626,3239.000000,-3.000000,-3.000000,0.000000,1.741248e+05,-3.000000,2016.0
50%,771.000000,391.000000,281.000000,650.500000,771.000000,771.000000,-3.000000,-3.000000,0.218125,42391.250000,...,0.145091,0.098183,0.281307,10542.000000,0.000000,0.000000,0.000000,2.290495e+05,-3.000000,2016.0
75%,2661.250000,1345.250000,1129.750000,2180.250000,2661.250000,2661.250000,0.043916,0.038351,0.354872,60715.800000,...,0.172916,0.141916,0.319785,26216.250000,0.000000,0.000000,0.000000,4.495985e+05,0.008010,2016.0
max,61975.000000,22902.000000,56938.000000,50030.000000,61975.000000,61975.000000,0.500000,0.559146,0.694932,152069.000000,...,0.434783,0.354839,0.612179,42580.000000,0.000000,0.000000,0.000000,4.877820e+07,0.346939,2016.0


In [213]:
sc_stud_char_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6976 entries, 0 to 6975
Data columns (total 51 columns):
age_24orolder_pct                   0 non-null object
count_family_income                 6976 non-null int64
count_family_income_dependent       6976 non-null int64
count_family_income_independent     6976 non-null int64
count_parent_ed                     6976 non-null int64
count_total_fafsa_applicants        6976 non-null int64
count_valid_dep_status              6976 non-null int64
dependent_highincome1_pct           6976 non-null float64
dependent_highincome2_pct           6976 non-null float64
dependent_highincome_pct            1921 non-null float64
dependent_income_mean               6976 non-null float64
dependent_lowincome_pct             6976 non-null float64
dependent_midincome1_pct            6976 non-null float64
dependent_midincome2_pct            6976 non-null float64
dependent_midincome_pct             3263 non-null float64
dependent_pct                       697

In [212]:
sc_stud_char_df.columns

Index(['age_24orolder_pct', 'count_family_income',
       'count_family_income_dependent', 'count_family_income_independent',
       'count_parent_ed', 'count_total_fafsa_applicants',
       'count_valid_dep_status', 'dependent_highincome1_pct',
       'dependent_highincome2_pct', 'dependent_highincome_pct',
       'dependent_income_mean', 'dependent_lowincome_pct',
       'dependent_midincome1_pct', 'dependent_midincome2_pct',
       'dependent_midincome_pct', 'dependent_pct', 'fafsa_sent_2ormore_pct',
       'fafsa_sent_3ormore_pct', 'fafsa_sent_4ormore_pct',
       'fafsa_sent_5ormore_pct', 'faminc_independents_mean', 'faminc_mean',
       'faminc_med', 'female_pct', 'fips', 'first_gen_student_pct',
       'highincome1_pct', 'highincome2_pct', 'highincome_pct',
       'independent_highincome1_pct', 'independent_highincome2_pct',
       'independent_highincome_pct', 'independent_income_mean',
       'independent_lowincome_pct', 'independent_midincome1_pct',
       'independent_midinc

In [236]:
sc_stud_char_columns = ['unitid', 'lowincome_pct', 'independent_pct', 
                        'dependent_lowincome_pct', 'independent_lowincome_pct', 
                        'midincome1_pct', 'midincome2_pct', 'highincome1_pct', 
                        'highincome2_pct', 'dependent_midincome1_pct', 
                        'dependent_midincome2_pct', 'dependent_highincome1_pct', 
                        'dependent_highincome2_pct', 'independent_midincome1_pct', 
                        'independent_midincome2_pct', 'independent_highincome1_pct', 
                        'independent_highincome2_pct', 'fafsa_sent_2ormore_pct', 
                        'fafsa_sent_3ormore_pct', 'fafsa_sent_4ormore_pct', 
                        'fafsa_sent_5ormore_pct', 'dependent_income_mean', 
                        'independent_income_mean', 'count_family_income', 
                        'count_family_income_dependent', 'count_family_income_independent', 
                        'count_valid_dep_status', 'count_parent_ed', 
                        'count_total_fafsa_applicants', 'female_pct', 'married_pct', 
                        'dependent_pct', 'veteran_pct', 'first_gen_student_pct', 
                        'faminc_mean', 'faminc_med', 'faminc_independents_mean', 
                        'midincome_pct', 'highincome_pct', 'dependent_midincome_pct', 
                        'dependent_highincome_pct', 'independent_midincome_pct', 
                        'independent_highincome_pct']
sc_stud_char = sc_stud_char_df[sc_stud_char_columns]

# Replace nan values with None
# Check if in directory
sc_stud_char = sc_stud_char.where((pd.notnull(sc_stud_char)), None)
sc_stud_char = sc_stud_char[sc_stud_char['unitid'].isin(ipeds_dir['unitid'])]

## Insert into MySQL Table

In [239]:
sc_stud_char_tuple = data_to_tuples(sc_stud_char)
insert_data(sc_stud_char_tuple, 'sc_stud_char', sc_stud_char.columns)

INSERT INTO sc_stud_char (unitid, lowincome_pct, independent_pct, dependent_lowincome_pct, independent_lowincome_pct, midincome1_pct, midincome2_pct, highincome1_pct, highincome2_pct, dependent_midincome1_pct, dependent_midincome2_pct, dependent_highincome1_pct, dependent_highincome2_pct, independent_midincome1_pct, independent_midincome2_pct, independent_highincome1_pct, independent_highincome2_pct, fafsa_sent_2ormore_pct, fafsa_sent_3ormore_pct, fafsa_sent_4ormore_pct, fafsa_sent_5ormore_pct, dependent_income_mean, independent_income_mean, count_family_income, count_family_income_dependent, count_family_income_independent, count_valid_dep_status, count_parent_ed, count_total_fafsa_applicants, female_pct, married_pct, dependent_pct, veteran_pct, first_gen_student_pct, faminc_mean, faminc_med, faminc_independents_mean, midincome_pct, highincome_pct, dependent_midincome_pct, dependent_highincome_pct, independent_midincome_pct, independent_highincome_pct) VALUES (%s, %s, %s, %s, %s, %s, 

'inserted data'

In [117]:
url = "https://educationdata.urban.org/api/v1/college-university/ipeds/grad-rates-pell/2016/"
ipeds_grad_pell = get_educationdata(url)
ipeds_grad_pell_df = pages_to_one_df(ipeds_grad_pell)

expecting 43228 rows of data
appended page 2
appended page 3
appended page 4
appended page 5
appended page 6
appended page 7
appended page 8
appended page 9
appended page 10
appended page 11
appended page 12
appended page 13
appended page 14
appended page 15
appended page 16
appended page 17
appended page 18
appended page 19
appended page 20
appended page 21
appended page 22
appended page 23
appended page 24
appended page 25
appended page 26
appended page 27
appended page 28
appended page 29
appended page 30
appended page 31
appended page 32
appended page 33
appended page 34
appended page 35
appended page 36
appended page 37
appended page 38
appended page 39
appended page 40
appended page 41
appended page 42
appended page 43
appended page 44


In [194]:
ipeds_grad_pell_df.head()

,cohort_adj,cohort_rev,cohort_year,completers_150pct,completion_rate_150pct,exclusions,fed_aid_type,fips,institution_level,subcohort,unitid,year
0,24,24,2011,12,0.500,0,99,17,4,2,147536,2016
1,4,4,2011,0,0.000,0,3,27,4,99,440767,2016
2,137,137,2014,100,0.730,0,1,6,1,99,119553,2016
3,8,8,2014,8,1.000,0,99,9,3,99,417178,2016
4,139,139,2011,40,0.288,0,3,42,4,99,215275,2016


In [200]:
ipeds_grad_pell_df.sort_values(by = ['unitid'])

,cohort_adj,cohort_rev,cohort_year,completers_150pct,completion_rate_150pct,exclusions,fed_aid_type,fips,institution_level,subcohort,unitid,year
23862,60,60,2011,23,0.383,0,3,1,4,2,100654,2016
29920,667,667,2011,148,0.222,0,1,1,4,2,100654,2016
29710,112,112,2011,30,0.268,0,2,1,4,99,100654,2016
42613,60,60,2011,23,0.383,0,3,1,4,99,100654,2016
6100,667,667,2011,148,0.222,0,1,1,4,99,100654,2016
8095,839,839,2011,201,0.240,0,99,1,4,99,100654,2016
25833,839,839,2011,201,0.240,0,99,1,4,2,100654,2016
38549,112,112,2011,30,0.268,0,2,1,4,2,100654,2016
11504,794,795,2011,466,0.587,1,3,1,4,2,100663,2016
1758,582,582,2011,276,0.474,0,1,1,4,2,100663,2016


In [195]:
ipeds_grad_pell_df[ipeds_grad_pell_df['unitid'] == 147536]

,cohort_adj,cohort_rev,cohort_year,completers_150pct,completion_rate_150pct,exclusions,fed_aid_type,fips,institution_level,subcohort,unitid,year
0,24,24,2011,12,0.500,0,99,17,4,2,147536,2016
411,9,9,2011,4,0.444,0,3,17,4,2,147536,2016
2211,15,15,2011,8,0.533,0,1,17,4,99,147536,2016
14138,24,24,2011,12,0.500,0,99,17,4,99,147536,2016
19729,15,15,2011,8,0.533,0,1,17,4,2,147536,2016
32513,0,0,2011,0,NaN,0,2,17,4,2,147536,2016
36679,0,0,2011,0,NaN,0,2,17,4,99,147536,2016
37966,9,9,2011,4,0.444,0,3,17,4,99,147536,2016
